# Calculate the metrics for validate set

In [1]:
import os

niigz_files = os.listdir()
niigz_files.sort()
# niigz_files

groundtruth_files = []
predict_files = []
for niigz_file in niigz_files:
    if "groundtruth" in niigz_file:
        groundtruth_files.append(niigz_file)
    if "predict" in niigz_file:
        predict_files.append(niigz_file)

groundtruth_files.sort()
predict_files.sort()

case_names = []
for file in groundtruth_files:
    case_name = file.split("-groundtruth")[0]
    case_names.append(case_name)

case_names.sort()
print(case_names)

test_dict = dict()
for case_name in case_names:
    test_dict[case_name] = {}
    groundtruth_niigz = case_name + "-groundtruth.nii.gz"
    predict_niigz = case_name + "-predict.nii.gz"
    test_dict[case_name]["groundtruth"] = groundtruth_niigz
    test_dict[case_name]["predict"] = predict_niigz

print(len(test_dict))

['ATM_001_0000', 'ATM_024_0000', 'ATM_034_0000', 'ATM_041_0000', 'ATM_060_0000', 'ATM_061_0000', 'ATM_074_0000', 'ATM_075_0000', 'ATM_080_0000', 'ATM_150_0000', 'ATM_158_0000', 'ATM_163_0000', 'ATM_197_0000', 'ATM_245_0000', 'ATM_246_0000', 'ATM_260_0000', 'ATM_266_0000', 'ATM_271_0000', 'ATM_638_0000']
19


In [2]:
import SimpleITK as sitk
import numpy as np

def load_CT_scan_3D_image(niigz_file_name):
    itkimage = sitk.ReadImage(niigz_file_name)
    numpyImages = sitk.GetArrayFromImage(itkimage)
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
    return numpyImages, numpyOrigin, numpySpacing

def false_positive_rate_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    fp = np.sum(pred - pred * label) + smooth
    fpr = round(fp * 100 / (np.sum((1.0 - label)) + smooth), 3)
    return fpr

def false_negative_rate_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    fn = np.sum(label - pred * label) + smooth
    fnr = round(fn * 100 / (np.sum(label) + smooth), 3)
    return fnr

def sensitivity_calculation(pred, label):   #  identical to True-Positive-Rate
    sensitivity = round(100 - false_negative_rate_calculation(pred, label), 3)
    return sensitivity

def dice_coefficient_score_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    intersection = np.sum(pred * label)
    dice_coefficient_score = round(((2.0 * intersection + smooth) / (np.sum(pred) + np.sum(label) + smooth)) * 100, 2)
    return dice_coefficient_score

def precision_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    tp = np.sum(pred * label) + smooth
    precision = round(tp * 100 / (np.sum(pred) + smooth), 3)
    return precision


In [4]:
FPR_list = []
FNR_list = []
Sensitivity_list = []
Precision_list = []
DSC_list = []

for item in test_dict.items():
    case_name = item[0]
    gt_npy, origin, spacing = load_CT_scan_3D_image(item[1]["groundtruth"])
    pred_npy, _, _ = load_CT_scan_3D_image(item[1]["predict"])
    
    FPR = false_positive_rate_calculation(pred_npy, gt_npy)
    FNR = false_negative_rate_calculation(pred_npy, gt_npy)
    Sensitivity = sensitivity_calculation(pred_npy, gt_npy)
    Precision = precision_calculation(pred_npy, gt_npy)
    DSC = dice_coefficient_score_calculation(pred_npy, gt_npy)
    
    print("{0} & {1} & {2} & {3} & {4} & {5}"
          .format(case_name, FPR, FNR, Sensitivity, Precision, DSC))
    
    FPR_list.append(FPR)
    FNR_list.append(FNR)
    Sensitivity_list.append(Sensitivity)
    Precision_list.append(Precision)
    DSC_list.append(DSC)

print("mean & {0} & {1} & {2} & {3} & {4}"
      .format(np.mean(FPR_list), 
              np.mean(FNR_list), 
              np.mean(Sensitivity_list),
              np.mean(Precision_list),
              np.mean(DSC_list)))
    

ATM_001_0000 & 0.007 & 7.258 & 92.742 & 97.779 & 95.19
ATM_024_0000 & 0.036 & 6.387 & 93.613 & 89.696 & 91.61
ATM_034_0000 & 0.021 & 4.039 & 95.961 & 96.455 & 96.21
ATM_041_0000 & 0.055 & 4.486 & 95.514 & 85.247 & 90.09
ATM_060_0000 & 0.027 & 2.682 & 97.318 & 92.505 & 94.85
ATM_061_0000 & 0.034 & 3.152 & 96.848 & 90.471 & 93.55
ATM_074_0000 & 0.06 & 2.934 & 97.066 & 85.671 & 91.01
ATM_075_0000 & 0.027 & 3.944 & 96.056 & 92.649 & 94.32
ATM_080_0000 & 0.031 & 3.662 & 96.338 & 92.216 & 94.23
ATM_150_0000 & 0.085 & 3.853 & 96.147 & 79.511 & 87.04
ATM_158_0000 & 0.047 & 3.003 & 96.997 & 86.234 & 91.3
ATM_163_0000 & 0.039 & 3.726 & 96.274 & 91.247 & 93.69
ATM_197_0000 & 0.032 & 10.332 & 89.668 & 90.723 & 90.19
ATM_245_0000 & 0.036 & 0.568 & 99.432 & 84.172 & 91.17
ATM_246_0000 & 0.041 & 0.629 & 99.371 & 82.469 & 90.13
ATM_260_0000 & 0.019 & 1.144 & 98.856 & 93.23 & 95.96
ATM_266_0000 & 0.025 & 2.077 & 97.923 & 91.103 & 94.39
ATM_271_0000 & 0.055 & 1.238 & 98.762 & 85.151 & 91.45
ATM_638_0000

In [5]:
BD_list = [71.05,
           87.54,
           93.06,
           83.42,
           90.17,
           85.05,
           91.64,
           88.68,
           78.55,
           74.78,
           87.87,
           88.06,
           64.03,
           100  ,
           100  ,
           98.9 ,
           100  ,
           98.4 ,
           90.48]

TLD_list = [85.1 ,
            93.5 ,
            94.67,
            90.69,
            94.34,
            90.35,
            94.44,
            92.16,
            88.01,
            88.6 ,
            92.67,
            94.41,
            81.64,
            100  ,
            100  ,
            98.71,
            99.24,
            98.34,
            95.91]
print("{0} & {1}".format(np.mean(BD_list), np.mean(TLD_list)))

87.98315789473685 & 93.3042105263158


In [12]:
metrics_3DUNet_valset = np.array([
    [0.017, 9.222 , 90.778, 93.232, 91.99, 80.34, 88.48],
    [0.032, 3.876 , 96.124, 93.343, 94.71, 76.78, 85.61],
    [0.031, 3.571 , 96.429, 91.66 , 93.98, 82.51, 89.9 ],
    [0.03 , 5.757 , 94.243, 92.35 , 93.29, 72.85, 84.36],
    [0.034, 3.432 , 96.568, 91.148, 93.78, 84.04, 91.09],
    [0.019, 26.864, 73.136, 94.026, 82.28, 32.38, 47.19],
    [0.023, 9.419 , 90.581, 91.951, 91.26, 73.3 , 84.83],
    [0.05 , 55.899, 44.101, 84.808, 58.03, 21.5 , 39.1 ],
    [0.023, 2.217 , 97.783, 93.409, 95.55, 93.07, 95.73],
])

metrics_3DUNet_testset = np.array([
    [0.006,     7.24 ,       92.76 ,      98.089,       95.35,      68.95,      84.17],      
    [0.031,     7.236,       92.764,      90.916,       91.83,      84.18,      92.86],      
    [0.022,     3.705,       96.295,      96.251,       96.27,      90.75,      93.93],      
    [0.047,     4.808,       95.192,      87.189,       91.02,      82.38,      90.38],      
    [0.024,     2.534,       97.466,      93.12 ,       95.24,      88.03,      92.72],      
    [0.028,     3.121,       96.879,      91.897,       94.32,      86.15,      90.8 ],      
    [0.053,     3.838,       96.162,      87.038,       91.37,      80.49,      89.86],      
    [0.023,     4.633,       95.367,      93.65 ,       94.5 ,      81.13,      88.56],      
    [0.026,     4.067,       95.933,      93.183,       94.54,      76.9 ,      88.11],      
    [0.082,     3.902,       96.098,      80.036,       87.33,      72.12,      87.62],      
    [0.044,     3.461,       96.539,      86.953,       91.5 ,      80.75,      90.48],      
    [0.035,     4.301,       95.699,      91.958,       93.79,      83.23,      91.34],      
    [0.032,     9.234,       90.766,      90.81 ,       90.79,      59.71,      78.46],      
    [0.038,     0.309,       99.691,      83.496,       90.88,      100  ,      100  ],
    [0.04 ,     0.611,       99.389,      82.853,       90.37,      100  ,      100  ],
    [0.016,     1.041,       98.959,      94.227,       96.53,      98.34,      97.93],      
    [0.019,     2.351,       97.649,      93.112,       95.33,      99.38,      98.39],      
    [0.049,     1.044,       98.956,      86.544,       92.33,      97.6 ,      97.62],      
    [0.014,     3.152,       96.848,      95.199,       96.02,      90.48,      95.45],      
])

print(metrics_3DUNet_valset.shape, metrics_3DUNet_testset.shape)

avg_metrics_valset = []
for column in range(metrics_3DUNet_valset.shape[1]):
    data = metrics_3DUNet_valset[:, column]
    avg = np.mean(data)
    avg_metrics_valset.append(avg)
    
print(avg_metrics_valset)

(9, 7) (19, 7)
[0.028777777777777777, 13.36188888888889, 86.63811111111112, 91.76966666666667, 88.31888888888889, 68.53, 78.47666666666667]


In [13]:
avg_metrics_testset = []
for column in range(metrics_3DUNet_testset.shape[1]):
    data = metrics_3DUNet_testset[:, column]
    avg = np.mean(data)
    avg_metrics_testset.append(avg)

print(avg_metrics_testset)

[0.03310526315789474, 3.715157894736842, 96.28484210526317, 90.34321052631581, 93.1215789473684, 85.29315789473685, 92.03578947368422]
